<a href="https://colab.research.google.com/github/mahtabb95/tire/blob/main/without_GA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow scikit-learn pandas numpy


In [ ]:
from google.colab import files

import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
from sklearn.utils import class_weight

# Load uploaded file

df = pd.read_csv("/content/WA_Fn-UseC_-HR-Employee-Attrition.csv")

# Remove missing rows
df = df.dropna()

# Encode categorical variables
for col in df.select_dtypes(include=['object']).columns:
    df[col] = LabelEncoder().fit_transform(df[col])

# Extract features and target
X = df.drop("Attrition", axis=1).values
y = df["Attrition"].values

# Normalize features
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

# Compute class weights (for unbalanced data)
cw = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y),
    y=y
)
class_weights = {i: cw[i] for i in range(len(cw))}
print("Class Weights:", class_weights)


Class Weights: {0: np.float64(0.5961070559610706), 1: np.float64(3.1012658227848102)}


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

def create_base_model():
    model = Sequential()
    model.add(Dense(16, activation="relu", input_shape=(X.shape[1],)))
    model.add(Dense(8, activation="relu"))
    model.add(Dense(1, activation="sigmoid"))

    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
    model.compile(optimizer=optimizer, loss="binary_crossentropy")

    return model


In [ ]:
X_train_base, X_test_base, y_train_base, y_test_base = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

base_model = create_base_model()

base_model.fit(
    X_train_base, y_train_base,
    epochs=20, batch_size=32, verbose=0,
    class_weight=class_weights
)

base_probs = base_model.predict(X_test_base)


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 


In [ ]:
best_f1_base = 0
best_t_base = 0.5

for t in np.linspace(0.2, 0.8, 40):
    preds_t = (base_probs > t).astype(int)
    f1_t = f1_score(y_test_base, preds_t)
    if f1_t > best_f1_base:
        best_f1_base = f1_t
        best_t_base = t

print("🔥 Best threshold (Base ANN):", best_t_base)
print("🔥 Best F1 (Base ANN):", best_f1_base)


🔥 Best threshold (Base ANN): 0.5230769230769232
🔥 Best F1 (Base ANN): 0.5454545454545454


In [ ]:
base_preds = (base_probs > best_t_base).astype(int)

print("\n📊 Base ANN Performance (No GA):")
print("Accuracy :", accuracy_score(y_test_base, base_preds))
print("Precision:", precision_score(y_test_base, base_preds))
print("Recall   :", recall_score(y_test_base, base_preds))
print("F1-score :", f1_score(y_test_base, base_preds))



📊 Base ANN Performance (No GA):
Accuracy : 0.7959183673469388
Precision: 0.4235294117647059
Recall   : 0.7659574468085106
F1-score : 0.5454545454545454
